In [ ]:
# Q-network with experience replay

# import libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import deque
import random

# state
states = [0,1,2,3,4,5,6,7,8,9,10]
N_STATES = len(states)

# action
actions = [0,1,2,3] # left, right, up, down
N_ACTIONS = len(actions)

# transition probability
P = np.zeros((N_STATES, N_ACTIONS, N_STATES))

P[0,0,:] = [1,0,0,0,0,0,0,0,0,0,0]
P[0,1,:] = [0,1,0,0,0,0,0,0,0,0,0]
P[0,2,:] = [1,0,0,0,0,0,0,0,0,0,0]
P[0,3,:] = [0,0,0,0,1,0,0,0,0,0,0]

P[1,0,:] = [0.9,0,0,0,0.1,0,0,0,0,0,0]
P[1,1,:] = [0,0,0.9,0,0,0.1,0,0,0,0,0]
P[1,2,:] = [0,1,0,0,0,0,0,0,0,0,0]
P[1,3,:] = [0,1,0,0,0,0,0,0,0,0,0]

P[2,0,:] = [0,1,0,0,0,0,0,0,0,0,0]
P[2,1,:] = [0,0,0,0.9,0,0,0.1,0,0,0,0]
P[2,2,:] = [0,0,1,0,0,0,0,0,0,0,0]
P[2,3,:] = [0,0,0,0,0,0.9,0.1,0,0,0,0]

P[3,0,:] = [0,0,0,1,0,0,0,0,0,0,0]
P[3,1,:] = [0,0,0,1,0,0,0,0,0,0,0]
P[3,2,:] = [0,0,0,1,0,0,0,0,0,0,0]
P[3,3,:] = [0,0,0,1,0,0,0,0,0,0,0]

P[4,0,:] = [0,0,0,0,1,0,0,0,0,0,0]
P[4,1,:] = [0,0,0,0,1,0,0,0,0,0,0]
P[4,2,:] = [0.9,0.1,0,0,0,0,0,0,0,0,0]
P[4,3,:] = [0,0,0,0,0,0,0,0.9,0.1,0,0]

P[5,0,:] = [0,0,0,0,0,1,0,0,0,0,0]
P[5,1,:] = [0,0,0,0.1,0,0,0.8,0,0,0,0.1]
P[5,2,:] = [0,0.1,0.8,0.1,0,0,0,0,0,0,0]
P[5,3,:] = [0,0,0,0,0,0,0,0,0.1,0.8,0.1]

P[6,0,:] = [0,0,0,0,0,0,1,0,0,0,0]
P[6,1,:] = [0,0,0,0,0,0,1,0,0,0,0]
P[6,2,:] = [0,0,0,0,0,0,1,0,0,0,0]
P[6,3,:] = [0,0,0,0,0,0,1,0,0,0,0]

P[7,0,:] = [0,0,0,0,0,0,0,1,0,0,0]
P[7,1,:] = [0,0,0,0,0,0,0,0,1,0,0]
P[7,2,:] = [0,0,0,0,1,0,0,0,0,0,0]
P[7,3,:] = [0,0,0,0,0,0,0,1,0,0,0]

P[8,0,:] = [0,0,0,0,0.1,0,0,0.9,0,0,0]
P[8,1,:] = [0,0,0,0,0,0.1,0,0,0,0.9,0]
P[8,2,:] = [0,0,0,0,0,0,0,0,1,0,0]
P[8,3,:] = [0,0,0,0,0,0,0,0,1,0,0]

P[9,0,:] = [0,0,0,0,0,0,0,0,1,0,0]
P[9,1,:] = [0,0,0,0,0,0,0.1,0,0,0,0.9]
P[9,2,:] = [0,0,0,0,0,0.9,0.1,0,0,0,0]
P[9,3,:] = [0,0,0,0,0,0,0,0,0,1,0]

P[10,0,:] = [0,0,0,0,0,0.1,0,0,0,0.9,0]
P[10,1,:] = [0,0,0,0,0,0,0,0,0,0,1]
P[10,2,:] = [0,0,0,0,0,0.1,0.9,0,0,0,0]
P[10,3,:] = [0,0,0,0,0,0,0,0,0,0,1]

# reward
if True: # fuel-efficient robot
    R = -0.02 * np.ones((N_STATES, N_ACTIONS))
else: # fuel-inefficient robot
    R = -0.5 * np.ones((N_STATES, N_ACTIONS))

# discount factor
dis = 0.99

# set parameters ###############################################################
max_episodes = 4000
learning_rate = 1e-1
input_size = N_STATES
output_size = N_ACTIONS
# set parameters ###############################################################

# policy
random_policy = 0.25*np.ones((1, N_ACTIONS))
if False: # bad policy presented above (top)
    policy = np.zeros((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,0,1]
    policy[5,:] = [0,1,0,0]
    policy[6,:] = [0,1,0,0]
    policy[7,:] = [0,1,0,0]
    policy[8,:] = [0,1,0,0]
    policy[9,:] = [0,0,1,0]
    policy[10,:] = [0,0,1,0]
elif False: # random policy
    policy = 0.25*np.ones((N_STATES, N_ACTIONS))
elif True: # optimal policy presented above (bottom)
    policy = np.zeros((N_STATES, N_ACTIONS))
    policy[0,:] = [0,1,0,0]
    policy[1,:] = [0,1,0,0]
    policy[2,:] = [0,1,0,0]
    policy[3,:] = [0,1,0,0]
    policy[4,:] = [0,0,1,0]
    policy[5,:] = [0,0,1,0]
    policy[6,:] = [0,0,1,0]
    policy[7,:] = [0,0,1,0]
    policy[8,:] = [1,0,0,0]
    policy[9,:] = [1,0,0,0]
    policy[10,:] = [1,0,0,0]

# define a function - sample_action
def sample_action(policy_given_state):
    policy_now = policy_given_state
    cum_policy_now = np.cumsum(policy_now)
    random_coin = np.random.random(1)
    cum_minus_coin = cum_policy_now - random_coin
    return [n for n, i in enumerate(cum_minus_coin) if i > 0][0]

# define a function - sample_transition
def sample_transition(transition_prob_given_state_and_action):
    prob = transition_prob_given_state_and_action
    cum_prob = np.cumsum(prob)
    random_coin = np.random.random(1)
    cum_minus_coin = cum_prob - random_coin
    return [n for n, i in enumerate(cum_minus_coin) if i > 0][0]

# make a memory for a deque of maxlen 100 for experience replay
replay_meomory = deque(maxlen=100)

# define a function - one_hot
def one_hot(state):
    return np.identity(N_STATES)[state]

X = tf.placeholder(tf.float32, [None, input_size], name="input_x")

# First layer of weights
W1 = tf.get_variable("W1", shape=[input_size, output_size],
                     initializer=tf.contrib.layers.xavier_initializer())
Qpred = tf.matmul(X, W1)

# We need to define the parts of the network needed for learning a policy
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)

# Loss function
loss = tf.reduce_sum(tf.square(Y - Qpred))

# Learning
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Setting up our environment
init = tf.global_variables_initializer()
# tf.get_default_graph().finalize()
sess = tf.Session()
sess.run(init)

step_history = []
for episode in range(max_episodes):
    print(episode)
    e = 1. / ((episode / 10) + 1)
    step_count = 0
    s = np.random.choice([0,1,2,4,5,7,8,9,10]) # 3 and 6 removed
    s_one_hot = one_hot(s)
    done = False
    # print(s)
    # print(s_one_hot)

    # The Q-Network training
    while not done:
        x = np.reshape(s_one_hot, [1, input_size])
        # Choose an action by greedily (with e chance of random action) from
        # the Q-network
        Q = sess.run(Qpred, feed_dict={X: x})
        if np.random.rand(1) < e:
            a = sample_action(policy_given_state=random_policy)
        else:
            a = np.argmax(Q)
        # print(a)

        # Get new state and reward from environment
        s1 = sample_transition(transition_prob_given_state_and_action=P[s, a, :])
        s1_one_hot = one_hot(s1)
        if (s1 == 3):
            Q[0, a] = R[s, a] + dis * 1
            done = True
            step_history.append(1)
            # append current experience at the end of the deque and update the deque
            replay_meomory.append([s, a, Q, s1])
        elif (s1 == 6):
            Q[0, a] = R[s, a] + dis * (-1)
            done = True
            step_history.append(0)
            # append current experience at the end of the deque and update the deque
            replay_meomory.append([s, a, Q, s1])
        else:
            reward = R[s, a]
            x_next = np.reshape(s1_one_hot, [1, input_size])
            # Obtain the Q' values by feeding the new state through our network
            Q_next = sess.run(Qpred, feed_dict={X: x_next})
            Q[0, a] = reward + dis * np.max(Q_next)
            # append current experience at the end of the deque and update the deque
            replay_meomory.append([s, a, Q, s1])
            # if game is not over, continue playing game
            s = s1

        # Q-learning using experience replay
        sample = random.sample(replay_meomory, 1)
        #print(sample)
        x = np.reshape(one_hot(sample[0][0]), [1, input_size])
        Q = sample[0][2]
        # Train our network using target and predicted Q values on each episode
        sess.run(train, feed_dict={X: x, Y: Q})

history = np.cumsum(step_history) / (np.arange(max_episodes)+1)
plt.plot(history)
plt.show()